In [1]:
import numpy as np
import pandas as pd

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, GPT2LMHeadModel, GPT2Config, AdamW
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm

import warnings
warnings.filterwarnings('ignore')

/kaggle/input/mistral/pytorch/7b-instruct-v0.1-hf/1/config.json
/kaggle/input/mistral/pytorch/7b-instruct-v0.1-hf/1/pytorch_model-00002-of-00002.bin
/kaggle/input/mistral/pytorch/7b-instruct-v0.1-hf/1/tokenizer.json
/kaggle/input/mistral/pytorch/7b-instruct-v0.1-hf/1/tokenizer_config.json
/kaggle/input/mistral/pytorch/7b-instruct-v0.1-hf/1/pytorch_model.bin.index.json
/kaggle/input/mistral/pytorch/7b-instruct-v0.1-hf/1/pytorch_model-00001-of-00002.bin
/kaggle/input/mistral/pytorch/7b-instruct-v0.1-hf/1/special_tokens_map.json
/kaggle/input/mistral/pytorch/7b-instruct-v0.1-hf/1/.gitattributes
/kaggle/input/mistral/pytorch/7b-instruct-v0.1-hf/1/tokenizer.model
/kaggle/input/mistral/pytorch/7b-instruct-v0.1-hf/1/generation_config.json


In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
! pip install flash-attn --no-build-isolation

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 9.4 MB/s eta 0:00:00:00:010:01
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 2.3 MB/s eta 0:00:00
  Created wheel for flash-attn: filename=flash_attn-2.5.6-cp310-cp310-linux_x86_64.whl size=120337372 sha256=2ebbab84027cd3274ab85f6108f5b262ce6b4795e899e7a8bf10b5899df0a6bc
  Stored in directory: /root/.cache/pip/wheels/a8/1c/88/b959d6818b98a46d61ba231683abb7523b89ac1a7ed1e0c206
Successfully built flash-attn


In [4]:
model_path = "/kaggle/input/mistral/pytorch/7b-instruct-v0.1-hf/1"

tokenizer = AutoTokenizer.from_pretrained(model_path)
model_chat = AutoModelForCausalLM.from_pretrained(
    model_path,
    torch_dtype=torch.bfloat16,
    device_map="auto",
    trust_remote_code=True,
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [5]:
torch.backends.cuda.enable_mem_efficient_sdp(False)
torch.backends.cuda.enable_flash_sdp(False)

In [6]:
def generate_description(prompt):
    messages = [{
        "role":"user",
        "content": prompt
    }]

    model_inputs = tokenizer.apply_chat_template(messages, return_tensors = "pt").to(device)
    with torch.no_grad():
        generated_ids = model_chat.generate(
            model_inputs,
            max_new_tokens=7500,
            do_sample=True,
            temperature=0.7, 
            top_k=50, 
            top_p=0.95,
            pad_token_id = tokenizer.eos_token_id       
        )

    decoded = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)
    text = decoded[0].split("[/INST]")[1]
    return text

In [8]:
def product_generator(product_name):
    prompt = f'Напиши описание для товара {product_name} на русском языке'
    decsription = generate_description(prompt)

    return decsription

In [9]:
def usluga_generator(product_name):
    prompt = f'Напиши описание для товара {product_name} на русском языке'
    decsription = generate_description(prompt)

    return decsription

In [10]:
def work_generator(work_name):
    prompt = f'Напиши описание для товара {work_name} на русском языке'
    decsription = generate_description(prompt)

    return decsription

In [11]:
torch.cuda.empty_cache()

In [12]:
%%time

product_generator('Звуковая карта Steinberg UR12')

2024-04-05 20:56:17.831054: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-05 20:56:17.831165: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-05 20:56:18.011103: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


CPU times: user 32.9 s, sys: 933 ms, total: 33.9 s
Wall time: 39.6 s


' Steinberg UR12 это устройство, которое позволяет пользователям записывать и воспроизводить звуки. Устройство содержит 2 входа и 4 выхода для соединения с различными устройствами. Помимо этого, устройство содержит 8-трековой аудиоинтерфейс с возможностью записи и воспроизведения звуков в высококачественной форме. Устройство также содержит возможность записи и воспроизведения видео.\n\nУстройство содержит различные встроенные функции, такие как эффекты и контроллеры, которые позволяют пользователям добавлять эффекты и контроль к звукам. Устройство также содержит возможность соединения с компьютером, что позволяет пользователям записывать и редактировать звуки на компьютере.\n\nУстройство также содержит возможность соединения с мобильными устройствами, такими как смартфонами и таблицами, что позволяет пользователям записывать и воспроизводить звуки на этих устройствах.\n\nВ целом, Steinberg UR12 это удобное и качественное устройство, которое позволяет пользователям записывать и воспроиз